In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np 
import os
import pandas as pd

In [2]:
os.getcwd()
os.chdir('C://Users//Jaesung//Desktop//해오라기//합칠거')

In [3]:
data = pd.read_csv('전처리완료.csv',engine ='python')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18398 entries, 0 to 18397
Data columns (total 62 columns):
상권_코드              18398 non-null int64
서비스_업종_코드_명        18398 non-null int64
평균_영업_개월_수         18398 non-null float64
당월_매출_금액           18398 non-null int64
월요일_매출_금액          18398 non-null int64
화요일_매출_금액          18398 non-null int64
수요일_매출_금액          18398 non-null int64
목요일_매출_금액          18398 non-null int64
금요일_매출_금액          18398 non-null int64
토요일_매출_금액          18398 non-null int64
일요일_매출_금액          18398 non-null int64
시간대_00~06_매출_금액    18398 non-null int64
시간대_06~11_매출_금액    18398 non-null int64
시간대_11~14_매출_금액    18398 non-null int64
시간대_14~17_매출_금액    18398 non-null int64
시간대_17~21_매출_금액    18398 non-null int64
시간대_21~24_매출_금액    18398 non-null int64
남성_매출_금액           18398 non-null int64
여성_매출_금액           18398 non-null int64
청소년층_매출금액          18398 non-null int64
청년층_매출금액           18398 non-null int64
중년층_매출금액           18398 non-null int64
장년층_매출금액 

In [5]:
X = data[data.columns[data.columns!='과밀_지수_구간_코드']]
Y = data[data.columns[data.columns=='과밀_지수_구간_코드']]

In [6]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

## modeling - XGBoost

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.cross_validation import ShuffleSplit
from sklearn.model_selection import StratifiedKFold

C:\Users\Jaesung\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
import warnings
warnings.filterwarnings(action='ignore') 

### n fold validation

In [9]:
n_folds = 3

def cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    score = cross_validate(model, X_train, y_train, scoring=['accuracy'], cv=kf, return_train_score=True)
    return(score)


#    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))



### 아래 파라미터들중 하나 선택하여 진행 

In [10]:
#model_xgb = xgb.XGBClassifier(colsample_bytree=0.4603, gamma=0.0468, 
#                             learning_rate=0.05, max_depth=3, 
#                             min_child_weight=1, n_estimators=500,
#                             reg_alpha=0.4640, reg_lambda=0.9571, 
#                             subsample=0.48, silent=1,
#                             random_state =7, nthread = -1)

In [61]:
model_xgb = xgb.XGBClassifier(colsample_bytree= 0.6, gamma= 1.5, learning_rate= 0.05, max_depth= 5, min_child_weight= 5, n_estimators= 700, subsample= 1.0
                             ,random_state =7, nthread = -1)



In [11]:
cv(model_xgb)

{'fit_time': array([25.97356153, 24.7179687 , 25.73017955]),
 'score_time': array([0.33313012, 0.31514859, 0.31311083]),
 'test_accuracy': array([0.73195156, 0.73998136, 0.72600186]),
 'train_accuracy': array([0.81046132, 0.80875844, 0.81306778])}

In [12]:
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4603, gamma=0.0468, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=1, nthread=-1, objective='multi:softprob',
       random_state=7, reg_alpha=0.464, reg_lambda=0.9571,
       scale_pos_weight=1, seed=None, silent=1, subsample=0.48)

In [13]:
predictions  = model_xgb.predict(X_test)

In [14]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_test, predictions, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, predictions))

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))

F-Score: 0.742
Accuracy:  0.7418478260869565
             precision    recall  f1-score   support

        0.0       0.84      0.86      0.85      1883
        1.0       0.67      0.64      0.65      1751
        2.0       0.71      0.73      0.72      1886

avg / total       0.74      0.74      0.74      5520

[[1611  152  120]
 [ 201 1113  437]
 [ 117  398 1371]]


## kfold with grid search (많은시간 소요)

In [10]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic',
                    silent=True, nthread=1)




In [11]:
parameters = {
        'learning_rate': [0.05],
        'min_child_weight': [1, 5, 10],
        'gamma': [0.05, 0.5, 1, 1.5],
        'subsample': [0.5, 0.7, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7],
        'n_estimators': [400, 700, 1000],
        }

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(n_splits=3, shuffle=True), 
                   scoring='accuracy',
                   verbose=3, refit=True)



In [12]:
clf.fit(X_train, y_train)

#trust your CV!
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Acc score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 972 candidates, totalling 2916 fits


[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:  2.3min
[Parallel(n_jobs=5)]: Done 118 tasks      | elapsed: 14.3min
[Parallel(n_jobs=5)]: Done 278 tasks      | elapsed: 44.2min
[Parallel(n_jobs=5)]: Done 502 tasks      | elapsed: 83.6min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 130.1min
[Parallel(n_jobs=5)]: Done 1142 tasks      | elapsed: 220.1min
[Parallel(n_jobs=5)]: Done 1558 tasks      | elapsed: 292.6min
[Parallel(n_jobs=5)]: Done 2038 tasks      | elapsed: 369.2min
[Parallel(n_jobs=5)]: Done 2582 tasks      | elapsed: 471.7min
[Parallel(n_jobs=5)]: Done 2916 out of 2916 | elapsed: 535.6min finished


Acc score: 0.6924211834135735
colsample_bytree: 0.6
gamma: 1.5
learning_rate: 0.05
max_depth: 5
min_child_weight: 5
n_estimators: 700
subsample: 1.0


In [47]:
predictions  = clf.predict(X_test)

ValueError: feature_names mismatch: ['활성도_지표_구간_코드', '안정성_지표_구간_코드', '서비스_업종_코드_명_커피음료', '성장성_지표_구간_코드', '서비스_업종_코드_명_분식집', '서비스_업종_코드_명_패스트푸드점', '서비스_업종_코드_명_치킨집', '총_직장인구_수', '점포수'] ['활성도_지표_구간_코드', '안정성_지표_구간_코드', '서비스_업종_코드_명_커피음료', '성장성_지표_구간_코드', '서비스_업종_코드_명_분식집', '서비스_업종_코드_명_패스트푸드점', '서비스_업종_코드_명_치킨집', '총_직장인구_수', '총_직장인구_수.1', '점포수']
training data did not have the following fields: 총_직장인구_수.1

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_test, predictions, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, predictions))

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))